In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("data_baseline//train.csv")
test = pd.read_csv("data_baseline///test.csv")

In [16]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,486,0,3,"Lefebre, Miss. Jeannie",female,NaN,3,1,4133,25.4667,NaN,S
1,426,0,3,"Wiseman, Mr. Phillippe",male,NaN,0,0,A/4. 34244,7.2500,NaN,S
2,780,1,1,"Robert, Mrs. Edward Scott (Elisabeth Walton Mc...",female,43.0,0,1,24160,211.3375,B3,S
3,248,1,2,"Hamalainen, Mrs. William (Anna)",female,24.0,0,2,250649,14.5000,NaN,S
4,184,1,2,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0000,F4,S


In [ ]:
test.head()

### Датасеты train и test отличаются толькой одной колонкой - той, что надо предсказать.

### Посмотрим на количество пропусков в каждой колонке

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

### Вообще, isna находит не все пропуски.
### Например, если внезапно численная переменная вдруг становится строкой - это звоночек, что там есть какие-то проблемы.
### Поэтому рекомендуется у строк, которые не являются численными (int64, float64), выводить уникальные значения.
### Там и можно обнаружить пропуски

### Посмотрим на тип данных каждой из колонок

In [ ]:
train.info()

In [ ]:
train.describe()

In [14]:
# здесь есть ловукша, поищите
# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'IsSeniorCitizen'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'HasPartner',
    'TotalSpent',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols
target_col = 'Churn'

### В sklearn большинство моделей не обрабатывают категориальные признаки.
### А у нас они есть... поэтому вспоминаем первую практическую и обрабатываем категориальные признаки.
### Также, мы применяли one-hot encoding к категориальным признакам, потому что с label-encoding в случае 3 и более значений образовывались операции +- над категориями.
### Но вспомним, что после one-hot encoding мы получали бинарные данные, а некоторые из признаков и так уже бинарные.

### А еще можно обрабатывать числовые признаки. Можно же?

In [ ]:
# *имитируем процесс обработки данных*

In [15]:
x = train[num_cols]
y = train[target_col]

KeyError: "None of [Index(['ClientPeriod', 'MonthlySpending', 'IsSeniorCitizen'], dtype='object')] are in the [columns]"

Посмотрим на распеределение значения целевой переменной

In [11]:
y.hist()
plt.show()

NameError: name 'y' is not defined

### Можем заметить, что Y несбалансирован
### Поэтому предсказательную способность модели будем оценивать по метрике ROC-AUC.
### Так же воспользуемся воспользуем разделением данных на тренировочную и валидационную выборку

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.7)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

### Посмотрим на получившийся результат и построим график для ROC-AUC. Обратите внимаение, что мы предсказываем колонку 'Churn' с помощью метода predict_proba(), а почему не обычный predict() ответите на защите

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

In [ ]:
y_train_predicted_knn = knn.predict_proba(x_train)[:, 1]
y_val_predicted_knn = knn.predict_proba(x_val)[:, 1]

In [ ]:
def roc_auc_plot(y_train, y_train_predicted, y_val, y_val_predicted):
    train_auc = roc_auc_score(y_train, y_train_predicted)
    test_auc = roc_auc_score(y_val, y_val_predicted)

    plt.figure(figsize=(10,7))
    plt.plot(*roc_curve(y_train, y_train_predicted)[:2], label='train AUC={:.4f}'.format(train_auc))
    plt.plot(*roc_curve(y_val, y_val_predicted)[:2], label='test AUC={:.4f}'.format(test_auc))
    legend_box = plt.legend(fontsize='large', framealpha=1).get_frame()
    legend_box.set_facecolor("white")
    legend_box.set_edgecolor("black")
    plt.plot(np.linspace(0,1,100), np.linspace(0,1,100))
    plt.show()

In [ ]:
roc_auc_plot(y_train, y_train_predicted_knn, y_val, y_val_predicted_knn)

In [ ]:
# Модель обучили теперь можно и предсказать колонку 'Churn' для test

In [ ]:
# сохраним колонку с id, для файла с ответом
test_id = test['id']
test = test[num_cols]
y_test_predicted_knn = knn.predict_proba(test)[:, 1]

In [ ]:
def create_submission(id_col, y_pred):
    submission = pd.DataFrame({
        'id': id_col,
        'Churn': y_pred
    })
    return submission

In [ ]:
sub = create_submission(test_id, y_test_predicted_knn)

In [ ]:
sub

In [ ]:
# сохраним файл с ответом и загрузим его для на kaggle
sub.to_csv("data_baseline//sub.csv", index=False)

### Перейдем на наше соревнование и загрузим ответ
### Для этого нажимаем на Submit Predictions
### Загружаем файл с ответом
### Нажимаем make Submsission
### Показывается наш результат
### Переходим во вкладку Leaderboard, что бы посмотреть на место, которые мы заняли

![image.png](attachment:image.png)

![image.png](attachment:image.png)

### Советы для улучшения результата
1. Построить графики для данных и поискать закономерности
2. Нормальная предобработка признаков, отбор признаков, создание новых признаков
3. Попробовать другие модели
4. Перебор параметров для моделей
5. Погуглить про задачу и посмотреть, как люди подходят к решению задачи классификации